In [43]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pandas.io.json import json_normalize
from collections import Counter
from operator import itemgetter
import datetime

# disable column truncation
pd.set_option('display.max_columns', None)

# Data Import & Cleaning
The purpose of this project is to analyze data collected by CBC in order to better understands how Canadians consume digital media.

The first portion of this project is to import, examine and clean the raw data so that it can be utilized for data analysis.

## 1. Import Data
First thing we need to do is import the data into a pandas dataframe. 

In [44]:
def build_dataframe(input_file_location):
    """
    Given an input path, read in all json files, returning results in a single dataframe
    """

    # grab input files
    input_files = os.listdir(input_file_location)

    dataframes = []

    # for each input file
    for file in input_files:
        
        filename, ext = os.path.splitext(file)
       
        # we only import json files
        if ext == '.json':

            # read file & append to dataframe list
            temp_df = pd.read_json(input_file_location + file, lines=True)
            dataframes.append(temp_df)

    
    if not dataframes:
        raise ValueError('No json files were found to import.  Check your input path and try again.')

    # build final dataframe and return
    return pd.concat(dataframes)    

In [45]:
# build dataframe
df = build_dataframe('../data/raw/')

# view column information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105256 entries, 0 to 105255
Data columns (total 49 columns):
$insert_id                   105236 non-null object
$schema                      105256 non-null int64
adid                         0 non-null float64
amplitude_attribution_ids    6075 non-null object
amplitude_event_type         6066 non-null object
amplitude_id                 105256 non-null int64
app                          105256 non-null int64
city                         104294 non-null object
client_event_time            105256 non-null datetime64[ns]
client_upload_time           105256 non-null datetime64[ns]
country                      105225 non-null object
data                         105256 non-null object
device_brand                 6083 non-null object
device_carrier               5441 non-null object
device_family                104982 non-null object
device_id                    105256 non-null object
device_manufacturer          6083 non-null object
device

In [46]:
# sneak peak at data
df.head()

,$insert_id,$schema,adid,amplitude_attribution_ids,amplitude_event_type,amplitude_id,app,city,client_event_time,client_upload_time,country,data,device_brand,device_carrier,device_family,device_id,device_manufacturer,device_model,device_type,dma,event_id,event_properties,event_time,event_type,group_properties,groups,idfa,ip_address,is_attribution_event,language,library,location_lat,location_lng,os_name,os_version,paying,platform,processed_time,region,sample_rate,server_received_time,server_upload_time,session_id,start_version,user_creation_time,user_id,user_properties,uuid,version_name
0,caf8305b-40e4-41e0-8692-5d6af97ec682,12,NaN,None,None,39902418291,161406,Vancouver,2018-04-01 23:00:11.055,2018-04-01 23:00:11.214,Canada,{},None,None,Apple iPhone,26ba7925-0a01-44fa-a8b8-649e64256bf1R,None,iPhone,Apple iPhone,None,23,"{'content.subsection2': 'newfoundland', 'conte...",2018-04-01 23:00:11.040,LOADED,{},{},NaN,None,False,English,amplitude-js/3.4.0,NaN,NaN,Mobile Safari,11,NaN,Web,2018-04-02 02:59:19.631632,British Columbia,NaN,2018-04-01 23:00:11.199,2018-04-01 23:00:11.214613,1522623611030,None,2018-03-24 07:13:47.998,None,"{'location.tv': 'vancouver', 'location.radio':...",d6849258-3621-11e8-a390-02ff4e2ee778,None
1,bc90c524-8c0a-47f7-8e95-8569a80b111d,12,NaN,None,None,39902418291,161406,Vancouver,2018-04-01 23:00:54.188,2018-04-01 23:00:54.521,Canada,{},None,None,Apple iPhone,26ba7925-0a01-44fa-a8b8-649e64256bf1R,None,iPhone,Apple iPhone,None,24,"{'content.subsection2': 'newfoundland', 'conte...",2018-04-01 23:00:54.191,READ,{},{},NaN,None,False,English,amplitude-js/3.4.0,NaN,NaN,Mobile Safari,11,NaN,Web,2018-04-02 02:59:34.301549,British Columbia,NaN,2018-04-01 23:00:54.524,2018-04-01 23:00:54.527986,1522623611030,None,2018-03-24 07:13:47.998,None,"{'location.tv': 'vancouver', 'location.radio':...",df4bf35e-3621-11e8-a390-02ff4e2ee778,None
2,0c5fa67d-cc5b-481e-b92e-3a2d41c76fd4,12,NaN,None,None,45860439719,161406,Prince Albert,2018-04-01 23:01:00.505,2018-04-01 23:01:01.815,Canada,{'first_event': True},None,None,Apple iPhone,4c199084-aef5-420b-b00c-4fc9813d0c4fR,None,iPhone,Apple iPhone,None,4,"{'content.authenticated': False, 'content.titl...",2018-04-01 23:01:01.378,LOADED,{},{},NaN,None,False,English,amplitude-js/3.4.0,NaN,NaN,Mobile Safari,11,NaN,Web,2018-04-02 02:59:37.661565,Saskatchewan,NaN,2018-04-01 23:01:02.688,2018-04-01 23:01:02.711141,1522623660496,None,2018-04-01 23:01:01.378,None,"{'location.tv': 'regina', 'location.radio': 'r...",e141669e-3621-11e8-a390-02ff4e2ee778,None
3,4d97cd3c-8f7b-4c3f-a3ea-5086b776a26a,12,NaN,None,None,39902418291,161406,Vancouver,2018-04-01 23:02:01.392,2018-04-01 23:02:01.578,Canada,{},None,None,Apple iPhone,26ba7925-0a01-44fa-a8b8-649e64256bf1R,None,iPhone,Apple iPhone,None,25,"{'content.authenticated': False, 'content.titl...",2018-04-01 23:02:01.383,LOADED,{},{},NaN,None,False,English,amplitude-js/3.4.0,NaN,NaN,Mobile Safari,11,NaN,Web,2018-04-02 02:59:55.988732,British Columbia,NaN,2018-04-01 23:02:01.569,2018-04-01 23:02:01.580194,1522623611030,None,2018-03-24 07:13:47.998,None,"{'location.tv': 'vancouver', 'location.radio':...",ec354084-3621-11e8-a390-02ff4e2ee778,None
4,ab68c06c-2c08-426f-9b09-8fea6c0e6e6a,12,NaN,None,None,27878795979,161406,Spruce Grove,2018-04-01 23:02:07.081,2018-04-01 23:02:07.449,Canada,{},None,None,Apple iPhone,52525baa-0860-4c22-9d12-fc3d1f729b1eR,None,iPhone,Apple iPhone,None,52,"{'content.subsection2': 'newfoundland', 'conte...",2018-04-01 23:02:07.118,LOADED,{},{},NaN,75.158.159.52,False,English,amplitude-js/3.4.0,NaN,NaN,Mobile Safari,11,NaN,Web,2018-04-02 02:59:57.745847,Alberta,NaN,2018-04-01 23:02:07.486,2018-04-01 23:02:07.492253,1522623727051,None,2017-08-16 01:10:35.762,None,"{'location.tv': 'calgary', 'location.radio': '...",ed433c1a-3621-11e8-a390-02ff4e2ee778,None


## 2. Standardize Column Names
We first standardize column names based on the following guidelines:
* Column names will contain lower case characters
* Column names will contain characters only (no numbers or special characters)
* Column names will use an underscore to denote spaces 

In [47]:
def standardize_column_names(data):
    """
    Standardize dataframe column names by 
        a. converting to lower case
        b. converting periods and spaces to underscores
        c. removing special characters and numbers
    """

    # get column names
    cols = list(data.columns)
    
    new_cols = []
    
    # for each column name
    for col in cols:
        
        # convert to lowercase
        col_name = col.lower()
        
        # substitute period or space with underscore
        col_name = re.sub('[\. ]', '_', col_name)

        # remove special characters & numbers (keeping underscores)
        col_name = re.sub('[^a-z0-9_]', '', col_name)
        
        new_cols.append(col_name)
        
    # update column names
    data.columns = new_cols

In [48]:
# standardize column names
standardize_column_names(df)

## 3. Time Analysis
The original filename we received is `161406_2018-04-01_23#629.json`.  Based on the filename, we assume the data contained within the file represents events received during the 23rd hour on 2018-04-01.  This does not mean that the data within the file occurred within this time window.  

We will assume that **client_event_time** identifies when an event actually occurred.  For the purposes of our analysis, we will focus on events that occurred within the 23rd hour on 2018-04-01.  

We have data that falls outside of this window (both before and after the window).  We will remove any events that occurred outside of this time range in this section.

In [49]:
def print_time_range(col_name):
    """
        Simple helper function to print out the min and max date found within a datetime column
    """
    print('--- {} ---'.format(col_name))
    print('min: {}'.format(min(df[col_name])))
    print('max: {}'.format(max(df[col_name])))
    print()

In [50]:
# print min & max dates for each datetime column
for col in df.select_dtypes(include='datetime64').columns:
    print_time_range(col)

--- client_event_time ---
min: 2012-01-01 01:58:04.874000
max: 2018-04-28 23:31:58.960000

--- client_upload_time ---
min: 2012-01-01 01:58:04.894000
max: 2018-04-28 23:31:58.961000

--- event_time ---
min: 2018-02-02 20:03:01.607000
max: 2018-04-01 23:59:59.825000

--- processed_time ---
min: 2018-04-02 02:59:19.631632
max: 2018-04-02 08:45:23.988137

--- server_received_time ---
min: 2018-04-01 23:00:00.038000
max: 2018-04-01 23:59:59.831000

--- server_upload_time ---
min: 2018-04-01 23:00:00.050132
max: 2018-04-01 23:59:59.936547

--- user_creation_time ---
min: 2017-03-25 00:42:28.073000
max: 2018-04-01 23:59:59.825000



In [51]:
# have a peak at data that occurred before 2018-04-01 23:00
df[df.client_event_time < datetime.datetime(2018,4,1,23,0)][['client_event_time', 'client_upload_time', 'event_time', 'server_received_time']].sort_values('client_event_time').head()

,client_event_time,client_upload_time,event_time,server_received_time
97408,2012-01-01 01:58:04.874,2012-01-01 01:58:04.894,2018-04-01 23:36:33.855,2018-04-01 23:36:33.875
64971,2018-02-02 20:03:01.059,2018-04-01 23:14:05.201,2018-02-02 20:03:01.607,2018-04-01 23:14:05.749
93079,2018-02-05 15:14:06.147,2018-04-01 23:30:41.963,2018-02-05 15:14:06.385,2018-04-01 23:30:42.201
72250,2018-02-07 21:58:45.476,2018-04-01 23:28:52.603,2018-02-07 21:58:46.574,2018-04-01 23:28:53.701
59479,2018-02-13 14:55:11.333,2018-04-01 23:23:11.235,2018-02-13 14:55:12.833,2018-04-01 23:23:12.735


In [52]:
# have a peak at data that occurred after 2018-04-01 23:59:59
df[df.client_event_time >= datetime.datetime(2018,4,2,0,0)][['client_event_time', 'client_upload_time', 'event_time', 'server_received_time']].sort_values('client_event_time').head()

,client_event_time,client_upload_time,event_time,server_received_time
40479,2018-04-02 00:00:00.113,2018-04-02 00:00:00.123,2018-04-01 23:59:59.406,2018-04-01 23:59:59.416
36992,2018-04-02 00:00:00.140,2018-04-02 00:00:02.744,2018-04-01 23:55:34.220,2018-04-01 23:55:36.824
75941,2018-04-02 00:00:01.381,2018-04-02 00:00:01.382,2018-04-01 23:58:31.678,2018-04-01 23:58:31.679
95592,2018-04-02 00:00:02.120,2018-04-02 00:00:02.123,2018-04-01 23:59:52.847,2018-04-01 23:59:52.850
5098,2018-04-02 00:00:02.678,2018-04-02 00:00:02.678,2018-04-01 23:59:59.316,2018-04-01 23:59:59.316


In [53]:
def remove_data_outside_window(df):
    """
        Ensures dataframe contains only events that occurred within the defined window start & end. 
    """
    window_start = datetime.datetime(2018,4,1,23,0)
    window_end = datetime.datetime(2018,4,2,0,0)
    
    return df[(df.client_event_time >= window_start) & (df.client_event_time < window_end)]

In [54]:
# limit records based on time window
df = remove_data_outside_window(df)

## 4. Set DataFrame Index
Each record within out dataset requires an id to uniquely identify itself.  We are assuming that this index is represented by insert_id.  

**Our assumptions are as follows:**

* insert_id is a unique identifier for each record, correlating to time in which the record was added to the database
* we should assume that all valid records contain an insert_id
* we should assume that insert_ids cannot be duplicated

With these assumptions made, we address two issues within the dataset:
1. Null insert_ids
2. Duplicate insert_ids

### Null Insert_ids
From the initial column overview, we know that insert_id contains 20 null values.  Examining the 20 records, it is noted that these records are tagged with a specific version (3.2.6).  It appears that users who are assigned this version may have had something go wrong while collecting data resulting in insert_id (and many other columns) to be null.  

For now, we will remove records which have a null insert_id.  The removal of these records is completed at the end of this section. 

In [55]:
df[pd.isnull(df.insert_id)].head()

,insert_id,schema,adid,amplitude_attribution_ids,amplitude_event_type,amplitude_id,app,city,client_event_time,client_upload_time,country,data,device_brand,device_carrier,device_family,device_id,device_manufacturer,device_model,device_type,dma,event_id,event_properties,event_time,event_type,group_properties,groups,idfa,ip_address,is_attribution_event,language,library,location_lat,location_lng,os_name,os_version,paying,platform,processed_time,region,sample_rate,server_received_time,server_upload_time,session_id,start_version,user_creation_time,user_id,user_properties,uuid,version_name
2503,None,12,NaN,None,None,47341150343,161406,None,2018-04-01 23:12:12.247,2018-04-01 23:12:12.429,None,{},None,None,Apple iPhone,7ce49306-fd77-4cd4-b08d-6fd30733b679,None,iPhone 6s,Apple iPhone 6s,None,733795750,{'content.title': 'juno-country-album-of-the-y...,2018-04-01 23:12:12.247,STREAMED,{},{},NaN,None,False,None,http/1.0,NaN,NaN,ios,11.2.6,NaN,iOS,2018-04-02 03:59:57.970781,None,NaN,2018-04-01 23:12:12.429,2018-04-01 23:12:12.434814,-1,3.2.6,2018-03-31 20:04:54.098,None,{},4f1b8d9a-362a-11e8-b4c4-06dadb7b56be,3.2.6
4296,None,12,NaN,None,None,24663964411,161406,None,2018-04-01 23:00:25.764,2018-04-01 23:00:26.186,None,{},None,None,iPod touch 6th Gen,3bdcf04c-4928-4832-a36a-cd9345c16008,None,iPod touch 6th Gen,None,None,81492977,"{'content.title': 'the-70s', 'content.media.li...",2018-04-01 23:00:25.764,STREAMED,{},{},NaN,None,False,None,http/1.0,NaN,NaN,ios,11.2.6,NaN,iOS,2018-04-02 04:14:27.981923,None,NaN,2018-04-01 23:00:26.186,2018-04-01 23:00:26.229983,-1,3.2.1,2017-06-24 20:02:40.098,None,{},55b70164-362c-11e8-9f9e-0228cc86fd90,3.2.6
8498,None,12,NaN,None,None,24705505643,161406,None,2018-04-01 23:21:06.204,2018-04-01 23:21:06.527,None,{},None,None,"iPad mini 2 (Retina/2nd Gen, Wi-Fi/GSM CDMA)",6c1ffec4-85fc-4b9e-af8e-f9f28915472d,None,"iPad mini 2 (Retina/2nd Gen, Wi-Fi/GSM CDMA)",None,None,428037313,"{'content.title': 'essential-classics', 'conte...",2018-04-01 23:21:06.204,STREAMED,{},{},NaN,None,False,None,http/1.0,NaN,NaN,ios,10.2.1,NaN,iOS,2018-04-02 04:29:42.591504,None,NaN,2018-04-01 23:21:06.527,2018-04-01 23:21:06.533281,-1,3.2.1,2017-06-25 16:37:06.591,None,{},76da016e-362e-11e8-8782-02121b3d8a82,3.2.6
9626,None,12,NaN,None,None,24705505643,161406,None,2018-04-01 23:27:39.835,2018-04-01 23:27:40.271,None,{},None,None,"iPad mini 2 (Retina/2nd Gen, Wi-Fi/GSM CDMA)",6c1ffec4-85fc-4b9e-af8e-f9f28915472d,None,"iPad mini 2 (Retina/2nd Gen, Wi-Fi/GSM CDMA)",None,None,865957735,"{'content.title': 'jazz-masters', 'content.med...",2018-04-01 23:27:39.835,STREAMED,{},{},NaN,None,False,None,http/1.0,NaN,NaN,ios,10.2.1,NaN,iOS,2018-04-02 04:31:52.059603,None,NaN,2018-04-01 23:27:40.271,2018-04-01 23:27:40.280429,-1,3.2.1,2017-06-25 16:37:06.591,None,{},c40e2708-362e-11e8-8782-02121b3d8a82,3.2.6
17391,None,12,NaN,None,None,41729759019,161406,None,2018-04-01 23:00:55.438,2018-04-01 23:00:55.892,None,{},None,None,iPad Air (Wi-Fi),4722451e-b9fb-46a7-baaf-aa6e574f87f5,None,iPad Air (Wi-Fi),None,None,657312391,{'content.title': 'women-who-have-changed-musi...,2018-04-01 23:00:55.438,FAVOURITED,{},{},NaN,None,False,None,http/1.0,NaN,NaN,ios,11.2.6,NaN,iOS,2018-04-02 04:48:55.339281,None,NaN,2018-04-01 23:00:55.892,2018-04-01 23:00:56.011735,-1,3.2.4,2018-01-27 17:56:55.761,None,{},25f83100-3631-11e8-befd-069dc6b5e0f2,3.2.6


### Duplicate Insert_ids
We made the assumption that insert_id is a unique value.  Is it actually unique within the dataset?  Looking at the dataset, it is found that we actually have duplicate (or even triplets) of various records.  

This might be a glitch within processing data, where the system does not acknowledge receiving a record and therefore accepts a second (or third) re-try of the insertion.  

In any case, we need to remove these records.  We will assume that the first value inserted is valid, and remove any duplicates after the first insertion.  The removal of these values will be completed at the end of this section.  

In [56]:
# create a counter to look at freqency of insert_id
count_by_insertid = Counter(df.insert_id)

# filter list to only include ids that have a count greater than 1
insertid_freq = {x : count_by_insertid[x] for x in count_by_insertid if count_by_insertid[x] > 1 }

# how many insert_ids are duplicated?
print('Duplicated insert_ids: {}'.format(len(insertid_freq.keys())))

Duplicated insert_ids: 134


In [57]:
sum(insertid_freq.values())

291

In [58]:
def remove_duplicate_index(df):
    """
        Removes duplicate index_id records, retaining the first version of the index_id record encountered 
    """
    
    # set index to be insert_id
    df = df.set_index('insert_id')

    # remove duplicated insert_id, keeping the first id that was created
    return df[~df.index.duplicated(keep='first')]


### Set DataFrame Index Clean-up

In [59]:
# remove na values
df.dropna(subset=['insert_id'], how='all')

# remove duplicate index values
df = remove_duplicate_index(df)

## 5. Normalize Nested Elements
After the initial viewing of the data, we can see that several columns contain nested dictionaries.  If we are going to utilize any of these nested elements in our analysis, we first need to normalize the data. In particular, we will address the following fields:
* Group Properties
* Groups
* Data
* User Properties
* Event Properties


### Group Properties
We check to see if there are any values within the column - finding that the column contains empty dictionaries.  As a result, we remove the column.  The removal of this column will be completed at the end of this section. 

In [60]:
# check if any values are present
len(df[~(df.group_properties == {})])

0

### Groups
We check to see if there are any values within the column - finding that the column contains empty dictionaries.  As a result, we remove the column.  The removal of this column will be completed at the end of this section. 

In [61]:
# check if any values are present
len(df[~(df.groups == {})])

0

### Data
Examining the data column, we find dictionaries containing a flag to identify whether the current event is the user's 'first event' recorded.  We want to preserve this data and will merge it back into the main dataframe.   The removal of this column and incorporation of data elements will be completed at the end of this section. 

In [62]:
# check if any values are present
len(df[~(df.data == {})])

18147

In [63]:
# take a peak at the data
df.data.head()

insert_id
caf8305b-40e4-41e0-8692-5d6af97ec682                       {}
bc90c524-8c0a-47f7-8e95-8569a80b111d                       {}
0c5fa67d-cc5b-481e-b92e-3a2d41c76fd4    {'first_event': True}
4d97cd3c-8f7b-4c3f-a3ea-5086b776a26a                       {}
ab68c06c-2c08-426f-9b09-8fea6c0e6e6a                       {}
Name: data, dtype: object

In [64]:
def normalize_data_column(df):
    """
        normalize the data column, adding the emedded first event back into the main dataframe
    """

    # normalize data
    df_first_event = json_normalize(df.data)
    df_first_event = df_first_event.set_index(df.index)

    # set 'first_event = False' for cases where the value is not set
    df_first_event['first_event'] = [False if pd.isnull(e) else e for e in df_first_event.first_event]

    # add first_event into the main dataframe
    return df.merge(df_first_event, left_index=True, right_index=True)

### User Properties
User properties appear to include media location preferences for a given user.  This could be interesting information to investigate further. We will add several columns back to our main dataframe; populating missing values with 'unknown' for analysis purposes. The removal of this column and incorporation of data elements will be completed at the end of this section. 

In [65]:
# check if any values are present
len(df[~(df.user_properties == {})])

53341

In [66]:
def normalize_user_properties_column(df):
    
    # normalize the data
    df_user_properties = json_normalize(data=df.user_properties)
    df_user_properties = df_user_properties.set_index(df.index)
    
    # standardize column names
    standardize_column_names(df_user_properties)

    # add user_preferences into the main dataframe
    return df.merge(df_user_properties, left_index=True, right_index=True)


### Event Properties
Event properties appear to be further details related to the recorded event.  Looking at the column details, we find that a significant amount of data is missing.   We will add several columns back to our main dataframe; populating missing values with 'unknown' for analysis purposes. The removal of this column and incorporation of data elements will be completed at the end of this section. 

In [67]:
# check if any values are present
len(df[~(df.event_properties == {})])

97233

In [68]:
def normalize_event_properties_column(df):
    
    # normalize the data
    df_event_properties = json_normalize(data=df.event_properties)
    df_event_properties = df_event_properties.set_index(df.index)
    
    # standardize column names
    standardize_column_names(df_event_properties)

    # add event_properties into the main dataframe
    return df.merge(df_event_properties, left_index=True, right_index=True)


### Nested Elements Clean-up
With the nested columns reviewed, we can incorporate interesting data back into our main dataframe and drop unncessary columns.

In [69]:
# run normalization functions
df = normalize_data_column(df)
df = normalize_user_properties_column(df)
df = normalize_event_properties_column(df)

# drop unncessary columns
df.drop(['group_properties', 'groups', 'data', 'user_properties', 'event_properties'], axis=1, inplace=True)

## 6. Removing Columns Containing Missing Data
We need to ensure the data we are analyzing is representative of the population.  We will remove columns that are missing more than 20% of data for this analysis.  

In [71]:
def find_missing_data_cols(df, data_threshold = 20):
    """
        returns a list of column names that are missing more than the specified data_threshold
    """
    
    # create a dictionary to hold the column name & missing data percentage
    cols = {}

    # loop through each column and find the percentage of missing data
    for c in df.columns:
        cols[c] = df[c].isnull().sum() / len(df) * 100
    
    # Our missing data threshold is set to 20% by default
    del_cols = []

    # loop through columns and only keep those that are missing the set data threshold
    for k, v in sorted(cols.items(), key=itemgetter(1), reverse=True):
        if cols[k] > data_threshold:
            print('{0}: {1:.2f}'.format(k, cols[k]))
            del_cols.append(k)
    
    # return a list of columns to delete
    return del_cols

In [72]:
# find column names to remove
remove_cols = find_missing_data_cols(df)

adid: 100.00
idfa: 100.00
location_lat: 100.00
location_lng: 100.00
paying: 100.00
sample_rate: 100.00
app_version: 100.00
content_categories: 100.00
feature_session: 100.00
feature_timedelay: 100.00
feature_origin: 99.99
u_vf: 99.96
content_sport: 99.87
custom_engine: 99.81
content_region: 99.44
content_keywords_collections: 99.43
user_id: 99.42
cmfappid: 99.29
syndicate: 99.14
content_show: 98.91
app_pillar: 98.87
content_media_type: 98.83
experiment_id: 98.81
experiment_variant: 98.81
content_genre: 98.61
content_pubdate: 98.38
content_media_audiovideo: 98.37
content_media_liveondemand: 98.37
content_subsection4: 97.98
usertier: 97.37
device_carrier: 97.02
amplitude_event_type: 96.72
amplitude_attribution_ids: 96.71
device_brand: 96.71
device_manufacturer: 96.71
start_version: 96.71
version_name: 96.71
dma: 94.76
content_subsection3: 89.50
ip_address: 88.51
content_keywords_company: 87.64
content_keywords_tag: 87.52
feature_position: 85.13
feature_name: 85.10
content_keywords_organi

In [73]:
# Remove columns
df.drop(remove_cols, axis=1, inplace=True)

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100527 entries, caf8305b-40e4-41e0-8692-5d6af97ec682 to 9fd94874-f301-45b9-9cfd-9b625b6ae6e0
Data columns (total 34 columns):
schema                   100527 non-null int64
amplitude_id             100527 non-null int64
app                      100527 non-null int64
city                     99616 non-null object
client_event_time        100527 non-null datetime64[ns]
client_upload_time       100527 non-null datetime64[ns]
country                  100515 non-null object
device_family            100255 non-null object
device_id                100527 non-null object
device_model             100255 non-null object
device_type              98957 non-null object
event_id                 100527 non-null int64
event_time               100527 non-null datetime64[ns]
event_type               100527 non-null object
is_attribution_event     100527 non-null bool
language                 100524 non-null object
library                  100527 non-null obje

# 7. Classifying Device Information
We have plenty of information around the device used to consume content (device family, model, and type).  For this analysis we are going to bring this information up to a very high level - classifying the device_type as either a phone, tablet, or desktop/laptop (windows, mac, linux).  

We will use this new field named **device_class** for our analysis and discard the detailed device information for now. 

In [75]:
def classify_device_type(device_type):
    
    if device_type is None:
        return 'Unknown'
    elif device_type in ['Linux', 'Mac', 'Windows']:
        return 'Desktop/Laptop'
    elif ('tab' in device_type) or ('iPad' in device_type):
        return 'Tablet'
    else:
        return 'Phone'

In [76]:
# classify the device type
df['device_class'] = df.device_type.apply(lambda d: classify_device_type(d))

# remove remaining device columns
df.drop(['device_family', 'device_id', 'device_model', 'device_type'], axis=1, inplace=True)

## 8. Correct Language
We would like to perform some analysis on the language column.  After reviewing the data we find that some events contain a language set to 'C'.  Since this is not telling of the true language, we will convert the 'C' to null.

In [77]:
# convert 'C' to null
df['language'] = [np.nan if l == 'C' else l for l in df.language]

## 9. Final Output
We will output this to a csv file that can then be uploaded in our analysis notebook.

In [79]:
df.to_csv('../data/processed/processed_data.csv')